# ENCS5341 - Machine Learning  
## Assignment 1: Data Preprocessing & Exploratory Data Analysis (EDA)

## Environment Setup

In [ ]:
!python -m venv .venv
# and then .venv\Scripts\activate in the terminal

## Install all required packages

In [ ]:
!pip install numpy pandas matplotlib seaborn

## Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Step 1: Data Loading and Initial Inspection:

### Loading the data

In [2]:
data = pd.read_csv("../data/Customer_Data.csv")

### Inspecting first few rows

In [3]:
data.head()

,CustomerID,Age,Gender,Income,Tenure,ProductType,SupportCalls,ChurnStatus
0,CUST0000,59.0,0,151203.0,4.0,0,1.0,0
1,CUST0001,69.0,0,58332.0,6.0,1,9.0,0
2,CUST0002,46.0,1,149481.0,2.0,0,12.0,0
3,CUST0003,32.0,1,115937.0,1.0,1,13.0,0
4,CUST0004,60.0,0,103929.0,4.0,1,5.0,0


### Checking general information

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CustomerID    3500 non-null   object 
 1   Age           3325 non-null   float64
 2   Gender        3500 non-null   int64  
 3   Income        3328 non-null   float64
 4   Tenure        3325 non-null   float64
 5   ProductType   3500 non-null   int64  
 6   SupportCalls  3329 non-null   float64
 7   ChurnStatus   3500 non-null   int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 218.9+ KB


### Summary statistics

In [5]:
data.describe()

,Age,Gender,Income,Tenure,ProductType,SupportCalls,ChurnStatus
count,3325.000000,3500.000000,3.328000e+03,3325.000000,3500.000000,3329.000000,3500.000000
mean,43.606617,0.495714,1.406860e+05,5.041504,0.298857,10.078702,0.044857
std,14.931487,0.500053,4.333271e+05,2.571029,0.457822,21.735374,0.207020
min,14.000000,0.000000,2.503700e+04,0.000000,0.000000,1.000000,0.000000
25%,31.000000,0.000000,5.653025e+04,3.000000,0.000000,3.000000,0.000000
50%,43.000000,0.000000,8.953250e+04,5.000000,0.000000,7.000000,0.000000
75%,56.000000,1.000000,1.215025e+05,7.000000,1.000000,11.000000,0.000000
max,69.000000,1.000000,5.004849e+06,9.000000,1.000000,200.000000,1.000000


## Step 2: Handling Missing Data

### Number of null values in each column

In [12]:
data.isnull().sum()

CustomerID        0
Age               0
Gender            0
Income            0
Tenure          175
ProductType       0
SupportCalls      0
ChurnStatus       0
dtype: int64

### Handling missing values for Age
We did not think using the mean for the age is the best possible solutions so we went for what we find is the best possible one.

We find the mean of the age for males and females separated and assign each null age value with the mean depending on the gender.

In [7]:
meanAgeFemale = data[data["Gender"] == 1]["Age"].mean()
meanAgeMale = data[data["Gender"] == 0]["Age"].mean()

data.loc[data["Gender"] == 1, "Age"] = data.loc[data["Gender"] == 1, "Age"].fillna(meanAgeFemale)
data.loc[data["Gender"] == 0, "Age"] = data.loc[data["Gender"] == 0, "Age"].fillna(meanAgeMale)

### Handling missing values for SupportCalls
We noticed that the minimum value for SupportCalls is 1, so we assumed that if the value is null it means that no calls were made therefor fill it with zero.

In [9]:
data["SupportCalls"] = data["SupportCalls"].fillna(0)

### Handling missing values for Income 
Filling the missing income values with the median is not sufficient.

So we grouped customers in bins based on their age range and filled the null income values with the median income of each group

In [11]:
bins = [10, 20, 30, 40, 50, 60, 70]
labels = ["10-19", "20-29", "30-39", "40-49", "50-59", "60-69"]
data["GroupByAge"] = pd.cut(data["Age"], bins=bins, labels=labels, right=False)

medianPerGroup = data.groupby("GroupByAge")["Income"].median()

for group in medianPerGroup.index:
    data.loc[(data["GroupByAge"] == group) & (data["Income"].isnull()), "Income"] = medianPerGroup[group]

data.drop("GroupByAge", axis=1, inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_29608\500644253.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  medianPerGroup = data.groupby("GroupByAge")["Income"].median()


### Handling missing values for Tenure
We filled the missing values based on Churn status and predefined group ratios

When grouping the data by tenure, we notice two things:

        1. All people who churn have been with the company for less than two years
        2. People who don't churn are split among the three groups, we will fill the missing data according to the existing ratio:
            0-2 -> 0.175
            3-5 -> 0.35
            6-9 -> 0.475

In [17]:
bins = [0, 3, 6, 10]
labels = ['0-2', '3-5', '6-9']

data['TenureGroup'] = pd.cut(data['Tenure'], bins=bins, labels=labels, right=False)
table = pd.crosstab(data['TenureGroup'], data['ChurnStatus'])
data.drop("TenureGroup", axis=1, inplace=True)
table

ChurnStatus,0,1
TenureGroup,,
0-2,592,157
3-5,1171,0
6-9,1580,0


Using the ratios generated below, we can apply the same ratio to the missing values

In [18]:
tenureGroupData = pd.cut(data[(data['ChurnStatus'] == 0) & (data['Tenure'].notnull())]['Tenure'], bins=bins, labels=labels, right=False)
ratios = tenureGroupData.value_counts(normalize=True)
ratios

Tenure
6-9    0.472629
3-5    0.350284
0-2    0.177086
Name: proportion, dtype: float64

In [19]:
for i in data.index:
    if pd.isnull(data.loc[i, "Tenure"]):
        if data.loc[i, "ChurnStatus"] == 1:
            data.loc[i, "Tenure"] = np.random.uniform(0, 2)
        else:
            r = np.random.rand()
            if r < 0.175:
                data.loc[i, "Tenure"] = np.random.uniform(0, 2)
            elif r < 0.525:
                data.loc[i, "Tenure"] = np.random.uniform(3, 5)
            else:
                data.loc[i, "Tenure"] = np.random.uniform(6, 9)

### Number of null values in each column after solving missing values

In [20]:
data.isnull().sum()

CustomerID      0
Age             0
Gender          0
Income          0
Tenure          0
ProductType     0
SupportCalls    0
ChurnStatus     0
dtype: int64